In [1]:
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
sc.settings.verbosity = 3
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import shapiro, norm
import os

%config InlineBackend.figure_format='retina'
%matplotlib inline
plt.rcParams['figure.figsize']=(7,4)


# Import files

In [2]:
adata = sc.read_h5ad('./xxx.h5ad')


# Cells neighboring (<=100um) tumor clusters are defined as peri-tumor 

In [1]:
# Spatial clusters are defined by Cellcharter (  1.ST_cluster.ipynb

In [4]:
samples = sorted(list(adata.obs['sample'].unique()))
neighborsNiche = {}

for sample in samples:
    print(sample) 
    ad1 = adata[adata.obs['sample']==sample].copy()

    center_ad = ad1[ad1.obs['spatial_cluster'].isin(['C6: Tumor (Biogenesis)', 'C5: Tumor'])] 
    locus_center = center_ad.obsm['spatial'] 
    locus_allcell = ad1.obsm['spatial']  
    from sklearn.neighbors import NearestNeighbors
    nn = NearestNeighbors(n_neighbors=300, algorithm='auto', metric = 'euclidean')
    nn.fit(locus_allcell)
    distances, indices = nn.kneighbors(locus_center )
    distances = distances*0.5 

    ad1_iloc = {i:ad1.obs.index[i] for i in range(ad1.n_obs)}
    df = pd.DataFrame(np.where(distances > 100, None, indices)) 
    df['centerCellIdx'] = np.array(center_ad.obs.index) 
    df = df[df.isna().sum(axis=1) < 5] 
    df = df.melt(id_vars='centerCellIdx') 
    df = df.dropna() 
    df.columns = ['centerCellIdx', 'NeighIndex','ad1ObsIndex'] 
    df['ad1ObsIndice'] = np.array(df['ad1ObsIndex'].map(ad1_iloc)) 

    neighborsNiche[sample] = df


P0411_CC
P0411_LM
P0516_CC
P0516_LM
P0608_CC


IOStream.flush timed out


P0608_LM
P0725_CC
P0725_LM
P0907_CC
P0907_LM
P0914_CC
P0914_LM
P1019_CC
P1019_LM
P1121_CC
P1121_LM
P1128_CC
P1128_LM


In [5]:
neighborsNiche = pd.concat(neighborsNiche, ignore_index=True )


In [6]:
adata.obs['tissue_region'] = 'Adjacent-tissue'
adata.obs.loc[adata.obs.index.isin(set(neighborsNiche['ad1ObsIndice'])), 'tissue_region'] = 'Peri_tumor'
adata.obs.loc[adata.obs['spatial_cluster'].isin(['C6: Tumor (Biogenesis)', 'C5: Tumor']), 'tissue_region'] = 'Intra_tumor'
